## Objective

export csv of indonesian_state in polygon format

### Flow

1. search region https://nominatim.openstreetmap.org
2. on details copy the OSM ID (e.g 5802438)
3. paste the OSM ID into http://polygons.openstreetmap.fr/get_poly.py?id=5802438&params=0

### Output Format

``` json
{"type":"Feature","id":"JS","properties":{"name":"Jakarta Selatan"},"geometry":{"type":"Polygon","coordinates":[[[-101.90605,40.001626],[-95.306337,40.001626],[-95.207752,39.908518],[-94.884612,39.831841],[-95.109167,39.541563],[-94.983197,39.442978],[-94.824366,39.20747],[-94.610765,39.158177],[-94.616242,37.000263],[-100.087706,37.000263],[-102.042974,36.994786],[-102.053927,40.001626],[-101.90605,40.001626]]]}},
...
```

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib.request
import json


In [2]:
# provinces in indonesia
provinces = pd.read_csv('ID_provinces.csv', sep=',',header=None)
provinces.tail(5)

,0,1
29,76,SULAWESI BARAT
30,81,MALUKU
31,82,MALUKU UTARA
32,91,PAPUA BARAT
33,94,PAPUA


In [3]:
# get regencies in indonesia
regencies = pd.read_csv('ID_regencies.csv', sep=';', header=None, encoding="utf-8")
# regencies.tail(50)
regencies.shape[0]

# get column
# list(regencies)

514

In [4]:
regencies.columns=['regency_id','province_id','regency_desc','osm_id']
is_jakarta = regencies['regency_desc'].str.contains("JAKARTA")
reg_jakarta = regencies[is_jakarta]

reg_jakarta['osm_id'] = reg_jakarta['osm_id'].astype(int)
reg_jakarta = reg_jakarta[is_jakarta]
reg_jakarta.head(5)

/Users/rijdz/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/rijdz/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,regency_id,province_id,regency_desc,osm_id
155,3171,31,KOTA JAKARTA SELATAN,5802438
156,3172,31,KOTA JAKARTA TIMUR,5802441
157,3173,31,KOTA JAKARTA PUSAT,7625977
158,3174,31,KOTA JAKARTA BARAT,7626001
159,3175,31,KOTA JAKARTA UTARA,7626002


In [5]:
def get_polygon_coordinate(regency_id):
    # Extract Polygon Coordinate
    url = 'http://polygons.openstreetmap.fr/get_poly.py?id='+regency_id+'&params=0'
    req = urllib.request.Request(url, headers={'User-Agent' : "Magic Browser"})
    response = urllib.request.urlopen( req )
    output = response.read().decode(response.headers.get_content_charset())
    html = response.read()
    coordinate_raw = output.split("\n\t")    
    
    # len(coordinate_raw)
    coordinates = []
    for coordinate in coordinate_raw:
        line = coordinate.replace('\t',',')
        line = line.replace('\nEND\nEND\n','')
        splitted_line = line.split(',')
        if 'polygon' not in line:
            arr_line = []
            for each in splitted_line:
                each = float(each.rstrip())
                arr_line.append(each)
            coordinates.append(arr_line)
    
    return coordinates

cor = get_polygon_coordinate('5802438')
# print(cor)

## Handle Regency

In [6]:
features = []

for index,row in reg_jakarta.iterrows():
    feature = {}
    feature['type'] = 'Feature'
    feature['id'] = 'ID' + str(row['regency_id'])
    prop = {}
    prop['name'] = row['regency_desc'].replace('KOTA','')
    feature['properties'] = prop
    geometry = {}
    geometry['type'] = 'Polygon'
    coordinate = get_polygon_coordinate(str(row['osm_id']))
    geometry['coordinates'] = [coordinate]
    feature['geometry'] = geometry
    features.append(feature)

features_json = json.dumps(features)    
# print(features_json)

In [7]:
# Final
# {"type":"FeatureCollection","features":[features_json]}

jakarta_regencies = {}
jakarta_regencies['type'] = "FeatureCollection"
jakarta_regencies['features'] = features
jakarta_regencies_json = json.dumps(jakarta_regencies)

# print(jakarta_regencies_json)
#export json
with open('JK_regencies.json', 'w') as outfile:
    json.dump(jakarta_regencies, outfile)

## Result

output from code, e.g JK_regencies.json, will be used for folium to generate the map 